In [ ]:
!pip install -U agno google-genai

In [ ]:
import os

# Google GenAI API Key 설정
os.environ["GOOGLE_API_KEY"] = "{{API_KEY}}"

print("Environment variables set.")

In [ ]:
from agno.agent import Agent
from agno.models.google import Gemini

agent = Agent(
    model=Gemini(id="gemini-flash-lite-latest"),
    markdown=True,
)

# print_response() 대신 run()을 사용하여 응답 객체(RunResponse)를 직접 받습니다.
response = agent.run("2025년 기준 미국 대통령 누구야?")

# 응답 객체 내부의 텍스트 컨텐츠를 직접 출력합니다.
print(response.content)

In [ ]:
from agno.agent import Agent
from agno.team import Team
from agno.models.google import Gemini
from agno.tools.duckduckgo import DuckDuckGoTools

duckduckgo_tool = DuckDuckGoTools()

search_agent = Agent(
    model=Gemini(id="gemini-flash-lite-latest"),
    tools=[duckduckgo_tool],
    description="너는 세계 최고의 검색엔진 마스터야. 항상 duckduckgo 에서 최신 데이터를 가져와서 응답해야해. 단, 마지막에 항상 긁어온 duckduckgo의 주소를 알려주면서 출처를 줘야해",
    instructions=[
        "항상 한국어로만 검색해",
        "검색어는 적절한 3가지를 선택해서 3가지 각가 검색하고 종합해",
    ],
    role="검색결과 취합",
    markdown=True,
)

writer_agent = Agent(
    model=Gemini(id="gemini-flash-lite-latest"),
    description="너는 글쓰기 마스터야. 주어진 정보와 사용자의 요청을 기반으로 블로그 포스트를 작성해.",
    instructions=[
        "항상 한국어로만 작성해.",
        "글은 논문처럼 초록, 방법론, 연구, 결과 순으로 정리해.",
    ],
    role="글쓰기",
    markdown=True,
)

# 도구를 사용하여 검색 수행
team = Team(
    name="최신 데이터 기반 블로그 글쓰기 팀",
    members=[search_agent, writer_agent],
    model=Gemini(id="gemini-flash-lite-latest"),
    instructions="팀끼리 협업해서 유저가 원하는 정보를 검색하고 그에 기반해서 블로그 글을 작성하도록 만들어. 너가 업무를 위임해야해"
)

response = team.run("2025년 기준 미국 대통령에 최신정보를 바탕으로 블로그 글을 써")